In [1]:
#data sourcing
import sqlite3

#data manipulation
import pandas as pd

#preprocessing and modelling
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler


In [10]:
import sqlite3
import pandas as pd

# path to local database
PATH_TO_DB = '../raw_data/countrypop(2).db'

# connect to database
def get_data(country):
    '''This function get's the Data from a local Database
    and converts it to a Dataframe'''
    conn = sqlite3.connect(PATH_TO_DB)
    
    param = (country)  
    query = (f"""
                SELECT lon, lat, pop
                FROM {param}
                
                """)

    data = pd.read_sql_query(query.format(param), con=conn)  
    
    return data



In [12]:
get_data("ABW")

,lon,lat,pop
0,-70.063750,12.545417,14.850115
1,-70.063750,12.537083,10.358201
2,-70.055416,12.620417,0.000000
3,-70.055416,12.612083,16.689957
4,-70.055416,12.603750,49.780464
...,...,...,...
266,-69.872083,12.428750,5.795240
267,-69.872083,12.420417,10.164492
268,-69.872083,12.412083,2.496273
269,-69.863750,12.420417,1.479186


In [13]:
def predict(country, n_centers):
    """
    This function takes a country code (3 letters) and
    a number of centers n_centers.
    It returns a list of optimal coordinates.
    """
    # get df from queries.py
    df_country = get_data(country)
    
   
    # scaler on pop column
    scaler = MinMaxScaler()
    df_country.loc[:,'pop'] = scaler.fit_transform(df_country[['pop']])
    
    #fit the model
    kmeans = KMeans(n_clusters = n_centers,
                    max_iter = 300,
                    n_init = 10,
                    random_state = 0)
    kmeans.fit(X=df_country[['lat', 'lon']], sample_weight=df_country['pop'])
        

    # return center coordinates
    return kmeans.cluster_centers_


## Check ##
# to check your code, run:
# python -m Globalizer.predict
#if __name__=="__main__":
 #   print(predict(country = "ABW", n_centers = 5))


In [15]:
predict(1)

array([[ 12.51434546, -69.99408943]])

Trying to get a safer way of table_name substition in the query

In [48]:
import psycopg2

# path to local database
PATH_TO_DB = '../raw_data/countrypop(2).db'

# connect to database
def get_data_psy(table_name):
    '''This function get's the Data from a local Database
    and converts it to a Dataframe'''
    conn = psycopg2.connect(host="localhost",database=PATH_TO_DB)
    
    
    query = sql.SQL("""
                SELECT lon, lat, pop
                FROM {table_name}
                
                """).format(table_name=sql.Identifier(table_name))

    data = pd.read_sql_query(query, con=conn)  
    
    return data

In [46]:
get_data_psy("ABW")

OperationalError: connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
